In [6]:
import pandas as pd
import numpy as np
import xgboost as xgb
import warnings
warnings.simplefilter(action='ignore')

In [3]:
train = pd.read_csv('StudentPerformancePred.csv')
labels = pd.read_csv('train_labels.csv')

labels['session'] = labels.session_id.apply(lambda x: int(x.split('_')[0]) )
labels['q'] = labels.session_id.apply(lambda x: int(x.split('_')[-1][1:]) )

train['session_id'] = train['session_id'].astype(str)
labels['session'] = labels['session'].astype(str)

In [34]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score 
from sklearn.metrics import accuracy_score

## Loop through questions, create train & validation sets for each question
for q_no in range(1,19):

    ## Filter dataset based on current questio
    if q_no <= 3:
        select = train[train['level_group'] == '0-4']
    elif q_no <= 13:
        select = train[train['level_group'] == '5-12']
    else:
        select = train[train['level_group'] == '13-22']

    ## Create train & validation sets
    train_X, validate_X = train_test_split(select, test_size=0.2)
    train_y = labels.loc[(labels['session'].isin(train_X['session_id'])) & (labels['q'] == q_no)]
    validate_y = labels.loc[(labels['session'].isin(validate_X['session_id'])) & (labels['q'] == q_no)]

    train_X = train_X.drop(['session_id', 'level_group'], axis=1)
    train_y = train_y['correct']

    validate_X = validate_X.drop(['session_id', 'level_group'], axis=1)
    validate_y = validate_y['correct']

    ## Do modeling here
    model = xgb.XGBClassifier(eval_metric='error')
    model.fit(train_X, train_y)

    # y_pred = model.predict(validate_X)
    # print(cross_val_score(validate_y, y_pred, cv=5))

    print("Question", q_no, "accuracy:", np.mean(cross_val_score(model, validate_X, validate_y, cv=5)))

    

Question 1 accuracy: 0.6810914257024043
Question 2 accuracy: 0.9817526843227447
Question 3 accuracy: 0.9329517080825751
Question 4 accuracy: 0.7738160048451772
Question 5 accuracy: 0.5068933453111878
Question 6 accuracy: 0.7492035402215003
Question 7 accuracy: 0.6948877976733242
Question 8 accuracy: 0.5537902479550965
Question 9 accuracy: 0.6832154685434974
Question 10 accuracy: 0.4827120384191934
Question 11 accuracy: 0.5803142160471728
Question 12 accuracy: 0.8489279595094483
Question 13 accuracy: 0.6810970543934185
Question 14 accuracy: 0.6573333963746727
Question 15 accuracy: 0.5283186199350224
Question 16 accuracy: 0.7004007628002062
Question 17 accuracy: 0.6526606822423804
Question 18 accuracy: 0.9475921585579744
